In [122]:
import os
import sys
import pandas as pd
import re
import geopandas as gpd
import numpy as np
from collections import defaultdict

In [123]:
# Load molecule file
moleculeFilePath = r'/content/drive/MyDrive/element_matrix_P1A.csv'
moleculesAsDF = pd.read_csv(moleculeFilePath)
display(moleculesAsDF)

,Unnamed: 0,formula_isotopefree,mz,C,H,O,N,S,P,Cl,...,Protein,Tannin,Unsat HydroCarbon,O.C,DBE,AI,AI_mod,NOSC,CGFE,0
0,0,C10H10O4,193.552314,9.50,10.0,4.0,0.0,0.0,0.0,0.0,...,0,0,0,1.0,5.50,5.590909,0.555556,-0.210526,66.300000,C10H10O4
1,1,C10H10O5,209.547231,9.50,10.0,5.0,0.0,0.0,0.0,0.0,...,0,0,0,1.0,5.50,4.388889,0.555556,0.000000,60.300000,C10H10O5
2,2,C10H10O5S,241.017643,10.00,10.0,5.0,0.0,1.0,0.0,0.0,...,0,0,0,1.0,6.00,3.750000,0.529412,0.200000,54.600000,C10H10O5S
3,3,C10H10O7,241.035387,10.00,10.0,7.0,0.0,0.0,0.0,0.0,...,0,1,0,1.0,6.00,2.333333,0.578947,0.400000,48.900000,C10H10O7
4,4,C10H10O8,257.030316,10.00,10.0,8.0,0.0,0.0,0.0,0.0,...,0,1,0,1.0,6.00,0.500000,0.578947,0.600000,43.200000,C10H10O8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,518,C9H8O4,179.285817,8.75,8.0,4.0,0.0,0.0,0.0,0.0,...,0,0,0,1.0,5.75,4.907895,0.636364,0.000000,60.300000,C9H8O4
519,519,C9H8O5,195.280748,8.75,8.0,5.0,0.0,0.0,0.0,0.0,...,0,0,0,1.0,5.75,3.683333,0.636364,0.228571,53.785714,C9H8O5
520,520,C9H8O5S,227.001994,9.00,8.0,5.0,0.0,1.0,0.0,0.0,...,0,0,0,1.0,6.00,2.666667,0.600000,0.444444,47.633333,C9H8O5S
521,521,C9H8O7,227.019729,9.00,8.0,7.0,0.0,0.0,0.0,0.0,...,0,1,0,1.0,6.00,1.000000,0.647059,0.666667,41.300000,C9H8O7


In [124]:
# Code to parse molecular formula into a dict that maps atoms to numbers

def splitStringAndDropDelimiters(regex, inputString):
  toReturn = []
  splitString = re.split(regex, inputString)
  # Convert string to nums and drop empty strings
  for val in splitString:
    if val != "":
      toReturn.append(val)

  return toReturn

#Code to parse molecular formula into dictionary mapping atoms to numbers
def parseFormula(formulaString):
  toReturn = defaultdict(lambda: 0)
  if formulaString=="":
    print("WARNING: given empty formula string")
    return toReturn
  
  atomRegex = '[A-Z][a-z]?'
  numRegex = '[0-9]*'

  splitByAtomAndNums = re.findall(atomRegex+numRegex,formulaString)
  # print('splitByAtomAndNums:', splitByAtomAndNums)
  for atomWithNum in splitByAtomAndNums:
    # print('atomWithNum:', atomWithNum)
    atom = re.findall(atomRegex, atomWithNum)[0]
    num = re.findall(numRegex,atomWithNum)[1]
    # print('num:', num)
    if num == '': 
      num = '1'
    num = int(num)
    toReturn[atom] = num

  # #Get list of atoms
  # atoms = splitStringAndDropDelimiters('[0-9]+', formulaString)
  
  # #Get list of numbers
  # numbers = [int(x) for x in splitStringAndDropDelimiters('\D', formulaString)]

  # if len(atoms) != len(numbers):
  #   print("WARNING: I think the number of atoms and numbers are not equal in", formulaString)

  # #Zip lists together
  # for atom, num in zip(atoms, numbers):
  #   toReturn[atom] = num
  
  return toReturn


  # print(atom)
  # print(num)


In [125]:
class Molecule:
  def __init__(self, nameString):
    self.name = nameString
    self.nameDict = parseFormula(nameString)
    
    C = float(self.nameDict['C'])
    H = float(self.nameDict['H'])
    O = float(self.nameDict['H'])
    N = float(self.nameDict['N'])
    P = float(self.nameDict['P'])
    S = float(self.nameDict['S'])
    Se = float(self.nameDict['Se'])

    # Compute HC and OC

    # If carbon is 0 then HC and OC set to 0
    self.HC = 0
    self.OC = 0
    if C != 0:
      self.HC = H/C
      self.OC = O/C

    # DBE
    self.DBE = 1 + .5*(2*C - H + N + P)

    # Compute AIKoch; 0 if either num or den is 0
    AINum = 1 + C - O - S - Se - .5*(N + P + H)
    AIDen = C - O - N - S - Se - P
    self.AIKoch = 0
    if AINum > 0 and AIDen > 0:
      self.AIKoch = AINum / AIDen
    
  def satsRule(self, rule):
    return rule(self.nameDict)

In [126]:
# Generate all molecules from strings from the input dataframe
molStringList = moleculeDF['formula_isotopefree'].tolist()
allMolecules = []
for molString in molStringList:
  allMolecules.append(Molecule(molString))

In [137]:
# Specify rules

# g1__BC_CHO<-(em[AIKoch>0.6667 & (em$elS+em$elSe+em$elN+em$elP)==0,]) # BC_CHO, combustion-derived polycyclic aromates (PCAs), black carbon without any N, S or P
def BC_CHO(mol):
  d = mol.nameDict
  return mol.AIKoch > 0.6667 and d['S']+d['Se']+d['N']+d['P'] == 0

# g1a_BC_CHO_C15max<-(em[i<-(AIKoch>0.6667 & (em$elS+em$elSe+em$elN+em$elP)==0 & em$elC<15),]); g1to10vector[i]<-"g1a"   # BC_CHO_C14max, subgroup with C<15
def BC_CHO_C15max(mol):
  d = mol.nameDict
  return mol.AIKoch > 0.6667 and d['S']+d['Se']+d['N']+d['P'] == 0 and d['C'] <15

# g1b_BC_CHO_C15min<-(em[i<-(AIKoch>0.6667 & (em$elS+em$elSe+em$elN+em$elP)==0 & em$elC>=15),]); g1to10vector[i]<-"g1b" # BC_CHO_C15min, subgroup with C>=15
def BC_CHO_C15min(mol):
  d = mol.nameDict
  return mol.AIKoch > 0.6667 and d['S'] + d['Se'] + d['N']+d['P'] == 0 and d['C'] >= 15

# g2__BC_CHOX<-(em[i<-(AIKoch>0.6667 & (em$elS+em$elSe+em$elN+em$elP)>0),]); g1to10vector[i]<-"g2"   # BC_CHOX, black carbon with N, S or P 
def BC_CHOX(mol):
  d = mol.nameDict
  return mol.AIKoch > 0.6667 and d['S'] + d['Se'] + d['N']+d['P'] > 0

# g3__Polyphen<-(em[AIKoch>=0.5 & AIKoch<=0.6667,]) # polyphenols, soil-derived polyphenols and PCAs with aliphatic chains
def Polyphen(mol):
  d = mol.nameDict
  return mol.AIKoch >= 0.5 and mol.AIKoch <= .6667

# g3a_Polyphen_Orich<-(em[i<-(AIKoch>=0.5 & AIKoch<=0.6667 & OC>=0.5),]); g1to10vector[i]<-"g3a"   # subgroup, O-rich polyphenols
def Polyphen_Orich(mol):
  d = mol.nameDict
  return mol.AIKoch >= 0.5 and mol.AIKoch <= .6667 and mol.OC >= .5

# g3b_Polyphen_Opoor<-(em[i<-(AIKoch>=0.5 & AIKoch<=0.6667 & OC<0.5),]); g1to10vector[i]<-"g3b"   # subgroup, O-poor polyphenols
def Polyphen_Opoor(mol):
  d = mol.nameDict
  return mol.AIKoch >= 0.5 and mol.AIKoch <= .6667 and mol.OC < .5

# g4__HiUnsatPhen<-(em[AIKoch<0.5 & HC<1.5 & OC<0.9,]) # highly unsaturated compounds, soil-derived humics, phenolics
def HiUnsatPhen(mol):
  d = mol.nameDict
  return mol.AIKoch < 0.5 and mol.HC < 1.5 and mol.OC < .9

# g4a_HiUnsatPhen_Orich<-(em[i<-(AIKoch<0.5 & HC<1.5 & OC<0.9 & OC>0.5),]); g1to10vector[i]<-"g4a"   # subgroup, highly unsaturated O-rich
def HiUnsatPhen_Orich(mol):
  d = mol.nameDict
  return mol.AIKoch < 0.5 and mol.HC < 1.5 and mol.OC < .9 and mol.OC > .5

# g4b_HiUnsatPhen_Opoor<-(em[i<-(AIKoch<0.5 & HC<1.5 & OC<=0.5),]); g1to10vector[i]<-"g4b"   # subgroup, highly unsaturated O-poor
def HiUnsatPhen_Opoor(mol):
  d = mol.nameDict
  return mol.AIKoch < 0.5 and mol.HC < 1.5 and mol.OC <= .5

# g5__UnsatAliph<-(em[HC>=1.5 & HC<=2 & OC<0.9 & em$elN==0,]) # unsaturated aliphatic compounds
def UnsatAliph(mol):
  d = mol.nameDict
  return mol.HC >= 1.5 and mol.HC <= 2 and mol.OC < .9 and d['N'] == 0

# g5a_UnsatAliph_Orich<-(em[i<-(HC>=1.5 & HC<=2 & OC<0.9 & OC>0.5 & em$elN==0),]); g1to10vector[i]<-"g5a"   # subgroup, unsaturated aliphatic compounds O-rich
def UnsatAliph_Orich(mol):
  d = mol.nameDict
  return mol.HC >= 1.5 and mol.HC <= 2 and mol.OC < .9 and mol.OC > .5 and d['N'] == 0

# g5b_UnsatAliph_Opoor<-(em[i<-(HC>=1.5 & HC<=2 & OC<=0.5 & em$elN==0),]); g1to10vector[i]<-"g5b"   # subgroup, unsaturated aliphatic compounds O-poor
def UnsatAliph_Opoor(mol):
  d = mol.nameDict
  return mol.HC >= 1.5 and mol.HC <= 2 and mol.OC <= .5 and d['N'] == 0

# g6__SatFA_CHO<-(em[i<-(HC>2 & OC<0.9 & (em$elS+em$elSe+em$elN+em$elP)==0),]); g1to10vector[i]<-"g6"   # saturated fatty acids CHO, without N, S or P
def SatFA_CHO(mol):
  d = mol.nameDict
  return mol.HC > 2 and mol.OC < .9 and d['S']+d['Se']+d['N']+d['P'] == 0


# g7__SatFA_CHOX<-(em[i<-(HC>2 & OC<0.9 & (em$elS+em$elSe+em$elN+em$elP)!=0),]); g1to10vector[i]<-"g7"   # saturated fatty acids CHOX, with N, S or P
def SatFA_CHOX(mol):
  d = mol.nameDict
  return mol.HC > 2 and mol.OC < .9 and d['S']+d['Se']+d['N']+d['P'] != 0

# g8__Carbo_CHO<-(em[i<-(OC>=0.9 & (em$elS+em$elSe+em$elN+em$elP)==0),]); g1to10vector[i]<-"g8"   # carbohydrates, sugars CHO
def Carbo_CHO(mol):
  d = mol.nameDict
  return mol.OC >= .9 and d['S']+d['Se']+d['N']+d['P'] == 0

# g9__Carbo_CHOX<-(em[i<-(OC>=0.9 & (em$elS+em$elSe+em$elN+em$elP)!=0),]); g1to10vector[i]<-"g9"   # carbohydrates, sugars CHOX
def Carbo_CHOX(mol):
  d = mol.nameDict
  return mol.OC >= .9 and d['S']+d['Se']+d['N']+d['P'] != 0

# g10_Pep<-(em[i<-(HC>=1.5 & HC<=2 & OC<0.9 & em$elN!=0),]); g1to10vector[i]<-"g10"   # peptides (unsaturated aliphatic and with at least 1 N)
def Pep(mol):
  d = mol.nameDict
  return mol.HC >= 1.5 and mol.HC <= 2 and d['N'] != 0

# # the following groups are not exclusive

# g11_CRAM<-(em[DBE/em$elC>=0.3 & DBE/em$elC<=0.68 & DBE/em$elH>0.2 & DBE/em$elH<=0.95 & DBE/em$elO>=0.77 & DBE/em$elO<=1.75,]) # CRAMs (carboxylic-rich alicyclic molecules)
def CRAM(mol):
  d = mol.nameDict
  if d['C'] == 0 or d['H']==0 or d['O']==0:
    print('WARNING: dividing by 0 for CRAM rule for,',mol.name , ', will assume it is not satisfied')
    return False
  return mol.DBE/d['C'] >= .3 and mol.DBE/d['C'] <= .68 and mol.DBE/d['H'] > .2 and mol.DBE/d['H'] <= .95 and mol.DBE/d['O'] >= .77 and mol.DBE/d['O'] <= 1.75

# g12_DNA<-(em[em$elP>=2 & em$elN>=2,]) # tentatively assigned as DNA-fragments
def DNA(mol):
  d = mol.nameDict
  return d['P'] <= 2 and d['N'] >= 2

# g13_N<-(em[em$elN>=1,]) # containing N
def N(mol):
  d = mol.nameDict
  return d['N'] >= 1

# g14_S<-(em[em$elS>=1,]) # containing S
def S(mol):
  d = mol.nameDict
  return d['S'] >= 1

# g15_P<-(em[em$elP>=1,]) # containing P
def P(mol):
  d = mol.nameDict
  return d['P'] >= 1

# g16_NS<-(em[em$elN>=1 & em$elS>=1,]) # containing N and S
def NS(mol):
  d = mol.nameDict
  return d['N'] >= 1 and d['S'] >= 1

# g17_NP<-(em[em$elN>=1 & em$elP>=1,]) # containing N and P
def NP(mol):
  d = mol.nameDict
  return d['N'] >= 1 and d['P'] >= 1

# Second set of rules

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elN>0  & vk.data$elP>0 & vk.data$elS>0]<-8
def CHONSP(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['N'] > 0 and d['P'] > 0 and d['S'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elN>0  & vk.data$elP>0]<-6
def CHOSP(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['S'] > 0 and d['P'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elN>0  & vk.data$elS>0]<-5
def CHONP(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['N'] > 0 and d['P'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elS>0  & vk.data$elP>0]<-7
def CHONS(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['N'] > 0 and d['S'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elN>0]<-2
def CHOP(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['P'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elS>0]<-3
def CHOS(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['S'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0 & vk.data$elP>0]<-4
def CHON(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0 and d['N'] > 0

# vk.data$color[is.na(vk.data$color) & vk.data$elC>0 & vk.data$elH>0 & vk.data$elO>0]<-1
def CHO(mol):
  d = mol.nameDict
  return d['C'] > 0 and d['H'] > 0 and d['O'] > 0

allRules = []
allRules.append(BC_CHO)
allRules.append(BC_CHO_C15max)
allRules.append(BC_CHO_C15min)
allRules.append(BC_CHOX)
allRules.append(Polyphen)
allRules.append(Polyphen_Orich)
allRules.append(Polyphen_Opoor)
allRules.append(HiUnsatPhen)
allRules.append(HiUnsatPhen_Orich)
allRules.append(HiUnsatPhen_Opoor)
allRules.append(UnsatAliph)
allRules.append(UnsatAliph_Orich)
allRules.append(UnsatAliph_Opoor)
allRules.append(SatFA_CHO)
allRules.append(SatFA_CHOX)
allRules.append(Carbo_CHO)
allRules.append(Carbo_CHOX)
allRules.append(Pep)
allRules.append(CRAM)
allRules.append(DNA)
allRules.append(N)
allRules.append(S)
allRules.append(P)
allRules.append(NS)
allRules.append(NP)

allRules.append(CHONSP)
allRules.append(CHOSP)
allRules.append(CHONP)
allRules.append(CHONS)
allRules.append(CHOP)
allRules.append(CHOS)
allRules.append(CHON)
allRules.append(CHO)


In [138]:
#Apply all rules to all molecules
dictRulesToMol = dict()
for rule in allRules:
  molBools = []
  for mol in allMolecules:
    molBools.append(int(rule(mol)))
  
  # Sum up number of times rule satisfied
  totalCountForRule = sum(molBools)
  molBools.insert(0, totalCountForRule)
  dictRulesToMol[rule.__name__] = molBools

columnNames = molStringList.copy()
columnNames.insert(0, 'total_count')

ruleDF = pd.DataFrame.from_dict(dictRulesToMol, orient='index',
                       columns=columnNames)
display(ruleDF)

,total_count,C10H10O4,C10H10O5,C10H10O5S,C10H10O7,C10H10O8,C10H10O9,C10H12N2O4,C10H12O7S,C10H4O5,...,C9H6O5,C9H6O6,C9H6O7,C9H6O7S,C9H8O3,C9H8O4,C9H8O5,C9H8O5S,C9H8O7,C9H8O8
BC_CHO,32,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
BC_CHO_C15max,24,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
BC_CHO_C15min,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BC_CHOX,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Polyphen,60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Polyphen_Orich,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Polyphen_Opoor,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HiUnsatPhen,248,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
HiUnsatPhen_Orich,248,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
HiUnsatPhen_Opoor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
#Output DF to file
homepath = r'/content/drive/MyDrive'
fileName = "/moleculesSortedByRules.csv"
fileString = homepath + fileName
ruleDF.to_csv(fileString, encoding='utf-8', index=True)